<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"
alt="Rubin Observatory logo, a graphical representation of turning stars into data."> <br><br>
**Description:** Generate simple tables for the _Rubin Observatory DP1_ paper. <br>
**Authors:** James Mullaney <br>
**Last verified to run:** 2025-05-14 <br>
**LSST Science Pipelines version:** v29.0.0<br>
**Container size:** medium <br>

# Simple Table Generation
Some tables in the DP1 paper draw in a lot of data and necessitate their own notebooks. However, there are a number of relatively "simple" tables that can all be generated in a single notebook.

## Setup

In [1]:
from lsst.daf.butler import Butler
from collections import defaultdict
import numpy as np
import copy

## DP1 Butler

In [2]:
instrument = 'LSSTComCam'
collections = ['LSSTComCam/DP1/defaults', 
               'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260',
               'skymaps', ]
skymap = 'lsst_cells_v1'
butler = Butler("/repo/dp1",
                instrument=instrument, 
                collections=collections, 
                skymap=skymap)
registry = butler.registry
skymap = butler.get('skyMap', skymap=skymap)

## Tables

### Available dimensions

In [3]:
dimensionDescriptions = {
    'day_obs':'A day and night of observations that rolls over during daylight hours.',
    'visit':"A sequence of observations processed together; synonymous with ``exposure'' in DP1.", 
    'exposure':'A single exposure of all nine ComCam detectors.',
    'instrument':'The instrument name.',
    'detector':'A ComCam detector.',
    'skymap':'A set of tracts and patches that subdivide the sky into rectangular regions with simple projections and intentional overlaps.',
    'tract':'A large rectangular region of the sky.',
    'patch':'A rectangular region within a tract.',
    'physical_filter':'An astronomical filter',
    'band':'An astronomicial wave band.',
}

detectors = [record.id for record in list(registry.queryDimensionRecords('detector'))]
skymap = list(registry.queryDimensionRecords('skymap'))[0].name
patches = set([record.id for record in list(registry.queryDimensionRecords('patch', datasets='template_coadd'))])
dimensionValues = {
    'day_obs':'YYYYMMDD',
    'visit':'YYYYMMDD\\#\\#\\#\\#\\#',
    'exposure':'YYYYMMDD\\#\\#\\#\\#\\#',
    'instrument':'LSSTComCam',
    'detector':f'{min(detectors)} - {max(detectors)}',
    'skymap':f'\\texttt{{{skymap.replace('_','\\_')}}}',
    'tract':'See Table \\ref{tab:dp1_tracts}',
    'patch':f'{min(patches)} - {max(patches)}',
    'physical_filter': 'u_02, g_01, i_06, r_03, z_03, y_04',
    'band':'u, g, r, i, z, y',
}

In [13]:
with open("../tables/dp1_dimension_summary.tex", "w") as f:
    f.write(r"""%%%%% This table is auto generated from data, DO NOT EDIT
\begin{deluxetable}{lp{3.5cm}p{8cm}}
\caption{Descriptions of and valid values for the key data dimensions in DP1. YYYYMMDD signifies date and \# signifies a single 0-9 digit.
\label{tab:dp1_dimensions} }
\tablehead{
  \colhead{\textbf{Dimension}} & \colhead{\textbf{Format/Valid values}} & \colhead{\textbf{Description}}\\ 
}
\startdata
""")
    for dimension in dimensionValues:
        latexName = dimension.replace('_', '\\_')
        f.write(f'\\texttt{{{latexName}}}&{dimensionValues[dimension]}&{dimensionDescriptions[dimension]}\\\\\n')
    f.write(r"""\enddata
\end{deluxetable}
""")
f.close()

### Tracts covering each field

In [15]:
fields = defaultdict(set)
with butler.query() as base_query:
    processed_visit_query = base_query.join_dataset_search("visit_summary").where('visit > 0')
    for row in processed_visit_query.general(["tract", "visit"], "visit.target_name"):
        fields[row["visit.target_name"]].add(row["tract"])

In [16]:
with open("../tables/dp1_field_tracts.tex", "w") as f:
    f.write(r"""%%%%% This table is auto generated from data, DO NOT EDIT
\begin{deluxetable}{lp4.5cm}}
\caption{Tract coverage of each DP1 field. 
\label{tab:dp1_tracts}}
\tablehead{
  \colhead{\textbf{Field Code}} & \colhead{\textbf{Tract ID}} 
}
\startdata
""")
    for field in fields:
        if field == 'slew_icrs':
            continue
        latexName = field.replace('_', '\\_')
        tracts = f'{np.array2string(np.sort(list(fields[field])), separator=', ')}'
        f.write(f'{latexName}&{tracts.strip('[]')}\\\\\n')
    f.write(r"""\enddata
\end{deluxetable}
""")
f.close()

### Number of raw images per field and band

In [7]:
fields = defaultdict(list)
with butler.query() as base_query:
    processed_visit_query = base_query.join_dataset_search("raw")
    for row in processed_visit_query.general(["band","detector"], "visit.target_name"):
        fields[row["visit.target_name"]].append(row["band"])

In [8]:
bandCounts = {'u':0, 'g':0, 'r':0, 'i':0, 'z':0, 'y':0}
rawCounts = {}
for field in fields.keys():
    rawCounts[field] = copy.deepcopy(bandCounts)
    
for field in fields.keys():
    for band in bandCounts.keys():
        rawCounts[field][band] += np.sum(np.array(fields[field]) == band)
        if field == 'slew_icrs':
            rawCounts['ECDFS'][band] += np.sum(np.array(fields[field]) == band)

In [14]:
fieldOrder = ['47_Tuc', 'ECDFS', 'EDFS_comcam', 'Fornax_dSph', 'Rubin_SV_095_-25', 'Rubin_SV_38_7', 'Seagull']
bandTotalCounts = {'u':0, 'g':0, 'r':0, 'i':0, 'z':0, 'y':0}
with open("../tables/rawbreakdown.tex", "w") as f:
    f.write(r"""%%%%% This table is auto generated from data, DO NOT EDIT
\setlength{\tabcolsep}{6pt}  % default is 6pt
\begin{deluxetable}{lccccccc}
\tablecaption{Number of \texttt{rawImages} per field and band.
\label{tab:rawbreakdown} }

\tablehead{
  \colhead{\textbf{Field Code}} & \multicolumn{6}{c}{\textbf{Band}} & \textbf{Total}\\
  \cline{2-7}
   &u&g&r&i&z&y& 
}
\startdata
""")
    for fieldName in fieldOrder:
        if fieldName == 'slew_icrs':
            continue
        latexName = fieldName.replace('_', '\\_')
        f.write(f'{latexName}')
        total = 0
        for band in ['u','g','r','i','z','y']:
            total += rawCounts[fieldName][band]
            bandTotalCounts[band] += rawCounts[fieldName][band]
            f.write(f'&{rawCounts[fieldName][band]}')
        f.write(f'&{total}\\\\\n')
    f.write('\\cline{1-8}\n')
    f.write('Total')
    bandTotal = 0
    for band in ['u','g','r','i','z','y']:
        bandTotal += bandTotalCounts[band]
        f.write(f'&{bandTotalCounts[band]}')
    f.write(f'&{bandTotal}\\\\\n')
    f.write(r"""\enddata
\end{deluxetable}
""")
f.close()

## Number and primary dimensions of each type of dataset

In [10]:
required = registry.getDatasetType('visit_detector_table').dimensions.required

In [11]:
datasetTypes = {'raw':'raw',
                'visit_image':'visit_image',
                'deep_coadd':'deep_coadd',
                'template_coadd':'template_coadd',
                'difference_image':'difference_image',
                'Source':'source',
                'Object':'object',
                'ForcedSource':'object_forced_source',
                'DiaSource':'dia_source',
                'DiaObject':'dia_object',
                'ForcedSourceOnDiaObject':'dia_object_forced_source',
                'CCDVisit':'visit_detector_table',
                'SSObject':'ss_object',
                'SSSource':'ss_source',
                'Visit':'visit_table',
               }

In [12]:
with open("../tables/dp1_butler_datasets.tex", "w") as f:
    f.write(r"""%%%%% This table is auto generated from data, DO NOT EDIT
\setlength{\tabcolsep}{6pt}  % default is 6pt
\begin{deluxetable}{llcc}
\tablecaption{The name and number of each type of data product in the Butler and the dimensions required to identify a specific dataset.
\label{tab:butlerdatasets} }

\tablehead{
  \textbf{Data Product} & 
  \textbf{Name in Butler} & 
  \textbf{Required Dimensions} & 
  \textbf{Number in DP1}\
}
\startdata
""")

    for datasetType in datasetTypes:
        reqDims = registry.getDatasetType(datasetTypes[datasetType]).dimensions.required
        reqDimsString = f'{reqDims}'.strip('{}')
        if len(reqDims) == 0:
            reqDimsString = '--'
        nDatasets = len(list(registry.queryDatasets(datasetTypes[datasetType])))
        latexName = datasetType.replace('_', '\\_')
        latexButlerName = datasetTypes[datasetType].replace('_', '\\_')
        f.write(f'\\texttt{{{latexName}}}&\\texttt{{{latexButlerName}}}&{reqDimsString}&{nDatasets}\\\\\n')
    f.write(r"""\enddata
\end{deluxetable}
""")
f.close()